In [76]:
import numpy as np
from indices import *
import math

In [77]:
daten = np.load("test_daten.npy")[100]

In [96]:
def create_window(array_source, radius, agents_count):  
    ZEILEN = 17
    WALL = 2
    CRATE = 1
    FREE = 0
    COIN = -1
    PLAYER = 3
    BOMB = 4
    EXPLOSION = 5
    AGENT = 6
    INDEX_LENGTH = 176
    
    row_length = radius * 2 + 1
    array_target = np.zeros([agents_count, row_length, row_length])
    rewards = np.zeros([agents_count])
    
    #Add Player and Bomb coordinates
    startX = INDEX_LENGTH
    for l in range(agents_count):
        mx, my = index_to_x_y(int(daten[int(startX)]),ZEILEN,ZEILEN) # Koordinaten des Spielers, Mittelpunkt des Ausschnitts
        borderX = mx + radius
        borderY = my + radius
        
        start = INDEX_LENGTH
        for i in range(agents_count):
            agentX, agentY = index_to_x_y(int(daten[int(start)]),ZEILEN,ZEILEN)
            # Set agents' positions
            if not (agentX > borderX or agentY > borderY or agentX < (mx - radius) or agentY < (my - radius)):
                array_target[l,int(agentY - (my - radius)), int(agentX - (mx - radius))] = AGENT
                start = start

            
            
            # Write bombs
            start += 2
            if not daten[int(start)] == 0:
                bombeX, bombeY = index_to_x_y(int(daten[int(start)]),ZEILEN,ZEILEN)
                if not (bombeX > borderX or bombeY > borderY or bombeX < (mx - radius) or bombeY < (my - radius)):
                    start = start
                    array_target[l,int(bombeY - (my - radius)), int(bombeX - (mx - radius))] = BOMB
            start = start + 19 
        
        # Write rewards
        rewards[l] = daten[int(startX + 1)]
        
        # Write event
        temp_events = daten[int(startX + 4): int(startX + 8)]
        print(temp_events)      
        #print(np.argmax(temp_events))
        
        for i in range(int(math.pow(row_length,2))):

            xt = i % row_length
            yt = math.floor(i / (row_length))
            x = xt + (mx - radius)
            y = yt + (my - radius)
            

            if array_target[l, int(yt),int(xt)] == 0:
            
                #print(y,x)
                try:
                    index = x_y_to_index(int(x),int(y),ZEILEN,ZEILEN)
                    start = start
                    #print(int(y),int(x))
                    array_target[l, int(yt),int(xt)] = daten[int(index)]
                except Exception as e:
                    array_target[l,yt,xt] = WALL
                    #print("WALL")
                    start = start
            #else:
                #print("Kollision " + str(int(y)) + " " + str(int(x)) )
                #print(int(yt),int(xt))
                
        
        startX = startX + 21
        # print("---")
    return array_target, rewards


In [97]:

create_window(daten,1,4)


[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]


(array([[[4., 2., 0.],
         [0., 6., 0.],
         [0., 2., 1.]],
 
        [[0., 1., 0.],
         [2., 6., 2.],
         [0., 1., 0.]],
 
        [[2., 2., 2.],
         [2., 6., 0.],
         [2., 2., 1.]],
 
        [[2., 0., 2.],
         [0., 6., 0.],
         [2., 0., 2.]]]), array([1., 2., 1., 3.]))

In [13]:
def create_observation_vectors(array_source, window_radius, agents_count):
    window_vector = create_window(array_source, window_radius, agents_count)
    observation_vector = np.zeros([agents_count, window_vector[0].flatten().shape[0]])
    # TODO: Add more derived observations:
    # ...
    
    # concatenate observations so far and window observations
    for i in range(agents_count):
        observation_vector[i,:] = window_vector[i].flatten()
    return observation_vector,  ,reward_db

In [14]:
create_observation_vectors(daten, 2, 4)

array([[3., 1., 0., 2., 2., 0., 2., 0., 2., 2., 0., 0., 6., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 6., 1., 3., 2.,
        2., 1., 2., 0., 2., 2., 1., 0., 0.],
       [2., 2., 1., 1., 1., 2., 2., 1., 2., 1., 2., 2., 6., 0., 1., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 6., 2., 2., 0.,
        2., 1., 2., 2., 0., 1., 1., 2., 2.]])

In [98]:
daten[176:]

array([102.,   1.,  89.,  -1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  43.,
         2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
         0.,   1.,   0.,   0.,   0.,   0.,   1.,   1.,  34.,   3.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0., 101.])